# Weeks 5 and 6. Natural language processing

## Part 2. Topic modeling
In this notebook, you'll learn to:
* Practice importing PDFs and handling exceptions
* Using LDA for topic modeling
* Exploring different parameters in the search for a meaningful result

Natural language processing is an umbrella term for tools to do automated text analysis. Here, we will focus on two of the most commonly used methods:
* This notebook: Topic modeling (as in the [Han et al.](https://www.tandfonline.com/doi/full/10.1080/01944363.2020.1831401) and [Brinkley & Stahmer](https://journals.sagepub.com/doi/abs/10.1177/0739456X21995890) papers)
* Subsequent notebook: Sentiment analysis (as in the [Schweitzer](https://www.tandfonline.com/doi/full/10.1080/01944363.2014.980439) paper)

So far, we've got some text into Python from a PDF, cleaned it up, and split the string into sentences and words. We've also done simple word counts. Now, we'll see what topic modeling can do.

Topic modeling is a method to identify patterns in text documents. It's an *unsupervised* machine learning technique, i.e. it doesn't need to be "trained" on a dataset of known topics. Rather, it identifies topics from the ground up. 

Another way to conceptualize topic modeling is *dimensionality reduction*. We'll see this concept again in cluster analysis in a week or two. Suppose we have 1000 distinct words in a document, that we reduce to 10 topics. We are going from a 1000-dimensional space to a 10-dimensional space. Obviously, 10 dimensions are much easier to interpret.

There are several algorithms that implement topic modeling; we'll focus on Latent Dirichlet Allocation (LDA) (as in the Han et al. paper). We'll use the `gensim` library, although `sklearn` is a popular alternative. `gensim` is dedicated to NLP, while `sklearn` is a more general machine-learning library.

If you haven't already, install them from the Terminal or Anaconda Prompt (remember to change your environment first):

`conda install scikit-learn gensim pyLDAvis -c conda-forge`

To avoid some harmless-but-annoying deprecation warnings, you should also run ([see here for why this is an issue](https://githubmemory.com/repo/bmabey/pyLDAvis/issues/196)):

`conda install ipython=7.10 -c conda-forge` 

We'll start with looking at a selection of [Climate Action Plans (CAPs)](https://drive.google.com/open?id=1iBzqlzr11hMtghAhZAVeHs9jvGIbrq1o). These are some of the plans I analyzed in a project with Hillary Angelo, Key MacFarlane, and James Sirigotis on equity in CAPs. Thanks to these collaborators for permission to share this dataset.

Download all of these plans to a folder on your computer.

In [7]:
path = '/Users/saratohamy/Documents/GitHub/UrbanDataScience/weeks5-6/CAPs/' # change this to wherever you downloaded the file

# We'll use the `os` library to get a list of all the plans
import os
filelist = os.listdir(path)  # returns a list of all the files in this directory/folder
# You should get a list like this. If not, check your path!
filelist[:10]

['palm_springs-_climate_action_plan.pdf',
 'Hillsborough_GHG Emissions Inventory_Climate Action Plan.pdf',
 'Mill Valley Climate Action Plan.pdf',
 'Palo Alto_CPP.pdf',
 'Redwood City_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan_General Plan Policy_General Plan Implementation Measures.pdf',
 'Martinez_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan_General Plan Policy.pdf',
 'Emeryville_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan.pdf',
 'Merced_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan_General Plan Policy_General Plan Implementation Measures.pdf',
 'Cathedral City_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan_Energy Action Plan.pdf',
 'San Leandro_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan_General Plan Policy_General Plan Implementation Measures_Codes or Ordinances.pdf']

In [9]:
import PyPDF2
import re

# This is similar code to last class, but we put it in a function
def readPDF(filename):
    # note that the filename argument must include the path
    print(filename) 
    numPages = pdf.getNumPages()
    txt = ''
    for page in pdf.pages:
        txt += page.extractText() 
        
    txt = re.sub(r"[^A-z\s]", "", txt)  # use regex to remove punctuation, numbers, etc.
    txt = re.sub(r"\s+", " ", txt) # use regex to remove excess whitespace
    
    print('Loaded {} with {} pages and {} characters'.format(filename, numPages, len(txt)))

    # remember to close the file, now we are done with it
    f.close()
    return txt


# now the advantages of a function become clear!
caps = [readPDF(path+fn) for fn in filelist]

# let's look at some random parts just to make sure
print(caps[0][10000:10200])
print(caps[2][10000:10200])

/Users/saratohamy/Documents/GitHub/UrbanDataScience/weeks5-6/CAPs/palm_springs-_climate_action_plan.pdf


NameError: name 'pdf' is not defined

In [4]:
print ([readPDF(path+fn) for fn in filelist])

FileNotFoundError: [Errno 2] No such file or directory: '/Users/saratohamy/Documents/GitHub/UrbanDataScience/weeks5-6/CAPspalm_springs-_climate_action_plan.pdf'

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> You probably got an error message. How can we (i) figure out which file caused the problem, and (ii) fix the problem?
</div>

In [10]:
# solution
def readPDF(filename):
    # can also do this 
    #if not (filename.endswith('.pdf')):
        #return None
    try:
        # note that the filename argument must include the path
        f = open(filename, 'rb') # rb means "read binary"
        pdf = PyPDF2.PdfFileReader(f)
        numPages = pdf.getNumPages()
        txt = ''
        for page in pdf.pages:
            txt += page.extractText() 

        txt = re.sub(r"[^A-z\s]", "", txt)  # use regex to remove punctuation, numbers, etc.
        txt = re.sub(r"\s+", " ", txt) # use regex to remove excess whitespace

        print('Loaded {} with {} pages and {} characters'.format(filename, numPages, len(txt)))

        # remember to close the file, now we are done with it
        f.close()
    except:
        # after running this, it seems that we failed on system files
        # so we could also have just skipped anything that doesn't end with .pdf
        print('Failed on {}'.format(filename))
        txt = ''
    return txt


# now the advantages of a function become clear!
caps = [readPDF(path+fn) for fn in filelist] # or you can add this to right aftr "filelist" on this line: if fn.endswith('.pdf')]

# let's exclude the ones that returned an empty string
caps = [cap for cap in caps if cap!='']

Loaded /Users/saratohamy/Documents/GitHub/UrbanDataScience/weeks5-6/CAPs/palm_springs-_climate_action_plan.pdf with 76 pages and 144037 characters


Loaded /Users/saratohamy/Documents/GitHub/UrbanDataScience/weeks5-6/CAPs/Hillsborough_GHG Emissions Inventory_Climate Action Plan.pdf with 71 pages and 128175 characters
Loaded /Users/saratohamy/Documents/GitHub/UrbanDataScience/weeks5-6/CAPs/Mill Valley Climate Action Plan.pdf with 88 pages and 111631 characters
Loaded /Users/saratohamy/Documents/GitHub/UrbanDataScience/weeks5-6/CAPs/Palo Alto_CPP.pdf with 114 pages and 219701 characters
Loaded /Users/saratohamy/Documents/GitHub/UrbanDataScience/weeks5-6/CAPs/Redwood City_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan_General Plan Policy_General Plan Implementation Measures.pdf with 55 pages and 113956 characters
Loaded /Users/saratohamy/Documents/GitHub/UrbanDataScience/weeks5-6/CAPs/Martinez_GHG Emissions Inventory_GHG Reduction Plan_Climate Action Plan_General Plan Policy.pdf with 59 pages and 123168 characters
Loaded /Users/saratohamy/Documents/GitHub/UrbanDataScience/weeks5-6/CAPs/Emeryville_GHG Emissions Inventor

Now, as before, let's remove the stop words. We could have also done this through adding to the `readPDF` function, and making that return a list of words that exclude stop words.

At this point, we might want to lemmatize as well.

In [12]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

swords = [re.sub(r"[^A-z\s]", "", sword) for sword in stopwords.words('english')]

# we did this in two steps before, but it's simpler to avoid creating the intermediate list of words

# this is a nested list comprehension. In the outer list, we loop over CAPs
# in the inner list, we loop over words in that CAP
# word_tokenize(cap.lower()) is what its looping over
wordlists = [[word for word in word_tokenize(cap.lower()) if word not in swords] for cap in caps]
# the same thing as:
#for cap in caps:
    #wl2 = []
    #wl = word_tokenize(cap.lower())
    #for word in wl:
        #if word not in swords:
            #wl2.append(word)
    #wordlists.append(wl2)
print(wordlists[0][10000:10050]) # first climat action plan, lets take a look at a random 50 words
print(wordlists[1][10000:10050])

['materials', 'selection', 'sustainable', 'site', 'development', 'water', 'savings', 'palm', 'springs', 'climate', 'action', 'plan', 'measures', 'primary', 'component', 'climate', 'action', 'plan', 'measures', 'specific', 'short', 'longterm', 'policies', 'programs', 'actions', 'jurisdiction', 'carry', 'reduce', 'greenhouse', 'gas', 'emissions', 'megawatt', 'mw', 'one', 'million', 'watts', 'typical', 'power', 'plant', 'generates', 'mw', 'power', 'methane', 'ch', 'greenhouse', 'gas', 'traps', 'times', 'amount', 'heat']
['systems', 'phase', 'include', 'encouraging', 'requiring', 'graywater', 'dual', 'plumbing', 'new', 'construction', 'major', 'remodels', 'adoption', 'new', 'water', 'conservation', 'programs', 'assist', 'town', 'e', 'ffective', 'response', 'growing', 'concerns', 'drought', 'required', 'long', 'term', 'reduction', 'water', 'usage', 'e', 'water', 'conservation', 'programs', 'goals', 'require', 'town', 'conserve', 'estimated', 'gallons', 'per', 'day', 'hillsborough', 'water',

OK, now we are at the point where we can do topic modeling.

The `gensim` documentation is pretty thorough.

Note that there are lots of options. The most important are:
* `corpus`: the text. More on the format for this below
* `num_topics`: how many topics you want to identify
* `alpha`: the expected distribution of topics across documents (i.e., are topics concentrated in a few documents)
* `eta`:  (sometimes called `beta`): the expected distribution of words across topics (i.e., are words concentrated in a few topics) 

However, sensible defaults are provided. So normally, a good approach is to start with the defaults and adjust accordingly. 

In [14]:
import gensim
help(gensim.models.LdaMulticore)

Help on class LdaMulticore in module gensim.models.ldamulticore:

class LdaMulticore(gensim.models.ldamodel.LdaModel)
 |  LdaMulticore(corpus=None, num_topics=100, id2word=None, workers=None, chunksize=2000, passes=1, batch=False, alpha='symmetric', eta=None, decay=0.5, offset=1.0, eval_every=10, iterations=50, gamma_threshold=0.001, random_state=None, minimum_probability=0.01, minimum_phi_value=0.01, per_word_topics=False, dtype=<class 'numpy.float32'>)
 |  
 |  An optimized implementation of the LDA algorithm, able to harness the power of multicore CPUs.
 |  Follows the similar API as the parent class :class:`~gensim.models.ldamodel.LdaModel`.
 |  
 |  Method resolution order:
 |      LdaMulticore
 |      gensim.models.ldamodel.LdaModel
 |      gensim.interfaces.TransformationABC
 |      gensim.utils.SaveLoad
 |      gensim.models.basemodel.BaseTopicModel
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, corpus=None, num_topics=100, id2word=None, workers=

[The `gensim` documentation](https://radimrehurek.com/gensim/models/ldamulticore.html#module-gensim.models.ldamulticore) provides some simple examples that help with getting up and running. There's also many other examples online. 

Conceptually, we need to:
* Convert our text to a list of lists. The outer list is each plan, and the inner list is words in that plan
* Create a gensim Dictionary (basically, each word gets an integer id)
* Feed that to the `gensim.models.LdaModel` function

The tricky part here is getting the data into the format (e.g. a list of words, list of word ids, string) that `gensim` expects. I did this mostly by adapting the examples on the `gensim` website.

In [15]:
dictionary = gensim.corpora.Dictionary(wordlists)
corpus = [dictionary.doc2bow(wl) for wl in wordlists] # converting list into something gensim can deal with 
model = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=10, alpha = 0.9, eta=0.9)

The `model` object now contains the results of our LDA model. We can explore some of its attributes and functions. The [documentation](https://radimrehurek.com/gensim/models/ldamulticore.html#) is also fairly comprehensive.

In [16]:
# list of topics
model.show_topics()

[(0,
  '0.008*"emissions" + 0.008*"city" + 0.008*"energy" + 0.007*"climate" + 0.006*"plan" + 0.005*"action" + 0.005*"reduction" + 0.005*"use" + 0.004*"ghg" + 0.004*"water"'),
 (1,
  '0.008*"emissions" + 0.007*"energy" + 0.007*"city" + 0.006*"climate" + 0.005*"plan" + 0.004*"reduction" + 0.004*"action" + 0.004*"ghg" + 0.003*"use" + 0.003*"community"'),
 (2,
  '0.010*"emissions" + 0.009*"energy" + 0.009*"city" + 0.007*"climate" + 0.006*"plan" + 0.005*"action" + 0.005*"ghg" + 0.004*"reduction" + 0.004*"gas" + 0.004*"use"'),
 (3,
  '0.008*"emissions" + 0.008*"energy" + 0.008*"city" + 0.007*"climate" + 0.005*"plan" + 0.004*"ghg" + 0.004*"action" + 0.004*"use" + 0.004*"reduction" + 0.004*"gas"'),
 (4,
  '0.009*"energy" + 0.009*"emissions" + 0.008*"city" + 0.006*"climate" + 0.005*"plan" + 0.005*"ghg" + 0.005*"reduction" + 0.004*"waste" + 0.004*"action" + 0.004*"gas"'),
 (5,
  '0.009*"emissions" + 0.008*"energy" + 0.007*"city" + 0.007*"climate" + 0.006*"plan" + 0.005*"reduction" + 0.004*"actio

In [17]:
model.show_topic(0)

[('emissions', 0.008341916),
 ('city', 0.008233373),
 ('energy', 0.008226286),
 ('climate', 0.00712993),
 ('plan', 0.005923786),
 ('action', 0.0047822),
 ('reduction', 0.0047448957),
 ('use', 0.0045510805),
 ('ghg', 0.0040920908),
 ('water', 0.0037946743)]

The data format looks pretty straightforward. Each topic is a list of tuples of (word, weight). With some effort, we might be able to plot this ourselves. 

But a quick web search reveals that there is a [Python library specificially designed to visualize LDA outputs](https://pyldavis.readthedocs.io/en/latest/readme.html)! 

In [19]:
import pyLDAvis
import pyLDAvis.gensim_models   # note that in previous versions this was called pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.002508 -0.000230       1        1  47.128840
4      0.000811  0.000868       2        1  17.322212
2      0.000683  0.000228       3        1  11.959461
0      0.000538 -0.000300       4        1   9.880262
5      0.000233 -0.000385       5        1   6.077167
6     -0.001358  0.000805       6        1   2.546387
8     -0.000234 -0.000013       7        1   1.731258
3     -0.000333 -0.000089       8        1   1.598564
9     -0.000714 -0.000603       9        1   1.041375
1     -0.002134 -0.000280      10        1   0.714474, topic_info=             Term         Freq        Total Category  logprob  loglift
706     emissions  3914.000000  3914.000000  Default  30.0000  30.0000
722        energy  3926.000000  3926.000000  Default  29.0000  29.0000
346          city  3369.000000  3369.000000  Default  28.0000  28.0000
350       climate  2865.000000  2865.000000  Default  27.0000  27.0000
1578         plan  2564.000000  2564.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
1831  residential     5.937134   945.487115  Topic10  -6.2497  -0.1291
1663     programs     6.235764  1029.469171  Topic10  -6.2006  -0.1651
1695       public     5.844147   935.279566  Topic10  -6.2655  -0.1340
254      building     5.748995   913.629135  Topic10  -6.2819  -0.1270
1430          new     5.996466  1079.178921  Topic10  -6.2398  -0.2514

[806 rows x 6 columns], token_table=       Topic      Freq Term
term                       
2431       1  0.359856    [
2431       2  0.191536    [
2431       3  0.168320    [
2431       4  0.104474    [
2431       5  0.063845    [
...      ...       ...  ...
12011      1  0.431462   yv
12011      2  0.215731   yv
12011      3  0.107866   yv
12011      4  0.107866   yv
12011      5  0.107866   yv

[2882 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 5, 3, 1, 6, 7, 9, 4, 10, 2])

So this is promising. But it's immediately clear that there are some nonsense topics (especially number 2). 

Perhaps a first step is to (i) remove words of <3 letters, and (ii) identify the plan that has a lot of these

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Adapt our list comprehension below to exclude words of less than 3 letters. And then see if you can exclude the plan that has "xgkc" in it
</div>

In [ ]:
# this is our original code, consolidated in one cell for you to adapt
wordlists = [[word for word in word_tokenize(cap.lower()) if word not in swords] for cap in caps]

# you probably don't need to change any of these lines
dictionary = gensim.corpora.Dictionary(wordlists)
corpus = [dictionary.doc2bow(wl) for wl in wordlists]
model = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=10, alpha = 0.9, eta=0.9)
pyLDAvis.gensim_models.prepare(model, corpus, dictionary)

In [20]:
# solution
wordlists = [[word for word in word_tokenize(cap.lower()) if word not in swords and len(word)>2] for cap in caps]

# now find the errant plan
problem_plans = [wl for wl in wordlists if 'xgkc' in wl]
print(len(problem_plans))
print(problem_plans) 


1
[['bbcb', 'ebe', 'fbe', 'gfbe', 'abb', 'ibibjbb', 'dbbbjbib', 'kblbb', 'mibibjneeb', 'oofpjbq', 'oorpcbbbq', 'hstocpcq', 'stoipabeq', 'dstoipleq', 'ombpsbeujq', 'stoafpq', 'dlbbvb', 'dlbbvibib', 'dbb', 'kbb', 'jbblbbb', 'hweex', 'ibbabvvb', 'hkeecxblbbbbvb', 'ybb', 'weexblbbbbvfbb', 'eezx', 'ibib', 'x\\bbbb', 'xbgebe^k', 'xbbbvpabnq', 'hxbbb', 'xbbbj', 'avx\\b', 'bbb', 'davx\\b', 'bbb', 'avxlbabve', 'wavxblbb', 'kavxblbb', 'havxblbb', 'hembbibv', 'xl^', 'rt^', '^kd', 'pkhkqh', 'tttl', 'abebvebbvb', 'bebvgbbb', '_zjb^ib^zbb^', 'st^jl^', 'lb^', '^sb^_cbr^a', 'lvb', 'lba', 'f^jbb', '^jea^jrs^mab^^lb^biv^', 'ztft', 'yc^a', 'mbbib', 'a_r^_ctb^bv', '^bobv^lv^ar^fjb', 'rtbbbb', 'zbbbv', 'crb', 'fab', 'lib', 'lit', 'bbb', 'icabv', 'bbbf', 'bvfv', 'bbib', 'mozb', 'zbbgzb', 'zbbgzb', '_lb', 'ijb', 'lrlt', 'fab', 'brb', 'zbbgzb', 'bbib', 'zb^', 'bdlzb', 'zbbgzb', 'bdlzb', 'ifb', 'b^^', '^lb^b^lb^', '^_s', 'b^zsb^bbbb^f', '^c^', 'bj^bbavv^b^am', 'beeb', 'xfbe', 'btv', 'beev', 'bebe', 'ob^', 'bbb

In [25]:
# seems like we should drop it
print(len(wordlists))
wordlists = [wl for wl in wordlists if 'xgkc' not in wl]
(len(wordlists))

29


29

In [21]:
# now the rest of the code
dictionary = gensim.corpora.Dictionary(wordlists)
corpus = [dictionary.doc2bow(wl) for wl in wordlists]
model = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=10, alpha = 0.9, eta=0.9)
pyLDAvis.gensim_models.prepare(model, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.003971  0.000399       1        1  54.423050
0      0.002537 -0.000731       2        1  29.522411
4      0.000619  0.001215       3        1   5.314979
5      0.000739 -0.000141       4        1   4.597196
2      0.000049 -0.000819       5        1   1.742412
9     -0.000019 -0.000512       6        1   1.249932
1     -0.002603  0.000790       7        1   0.946725
6     -0.002649 -0.000999       8        1   0.809191
3     -0.001543  0.000144       9        1   0.798319
8     -0.001100  0.000652      10        1   0.595785, topic_info=            Term         Freq        Total Category  logprob  loglift
698    emissions  4134.000000  4134.000000  Default  30.0000  30.0000
714       energy  3796.000000  3796.000000  Default  29.0000  29.0000
342         city  3555.000000  3555.000000  Default  28.0000  28.0000
346      climate  3129.000000  3129.000000  Default  27.0000  27.0000
1543        plan  2537.000000  2537.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
1518         per     5.141478  1025.767048  Topic10  -6.1730  -0.1728
380   commercial     4.705840   908.753581  Topic10  -6.2615  -0.1402
1660      public     4.771600   943.503845  Topic10  -6.2476  -0.1639
679   efficiency     4.899464  1056.671818  Topic10  -6.2212  -0.2507
489         cost     4.825423  1011.290808  Topic10  -6.2364  -0.2220

[830 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
9767       1  0.483423      _qxr
9767       2  0.241711      _qxr
9773       1  0.474740       abb
9773       2  0.237370       abb
2387       1  0.616741  accessed
...      ...       ...       ...
2371       9  0.006504      year
2371      10  0.005575      year
11647      1  0.558883       ytv
11647      2  0.335330       ytv
11647      3  0.111777       ytv

[2519 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 1, 5, 6, 3, 10, 2, 7, 4, 9])

It looks like we still have a problem with topics 7 and 8. And we have all the interesting stuff in topics 1 and 2.

This is where exploratory analysis comes in. We probably want to adjust the number of topics and the parameters, until we find a set of topics that makes intuitive sense.

If we were doing regression analysis, this would be "fishing" (bad!). But in unsupervised machine learning, this type of exploration is an inherent part of the process. We aren't testing hypotheses, just searching for patterns and understanding.

Remember that the `LdaMulticore` function takes the `num_topics`, `alpha` and `eta` parameters. We could adjust those. We might also want to exclude certain uninformative words based on our present context — perhaps "city" or "climate"

In [24]:
# this is our original code, consolidated in one cell for you to adapt
wordlists = [[word for word in word_tokenize(cap.lower()) if word not in swords and len(word)>2] for cap in caps]
wordlists = [wl for wl in wordlists if 'xgkc' not in wl]

dictionary = gensim.corpora.Dictionary(wordlists)
corpus = [dictionary.doc2bow(wl) for wl in wordlists]
model = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=10, alpha = 0.9, eta=0.9)
pyLDAvis.gensim_models.prepare(model, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.002773  0.001055       1        1  28.388316
4     -0.001556 -0.005077       2        1  26.643607
9     -0.002382  0.000665       3        1  15.103818
5     -0.003053  0.001196       4        1  13.474615
8      0.000698 -0.000078       5        1   9.393784
2     -0.001008 -0.000588       6        1   2.776429
6      0.002077 -0.001889       7        1   1.619188
3      0.000794  0.001413       8        1   1.076350
0      0.006891  0.000227       9        1   0.873142
1      0.000311  0.003076      10        1   0.650750, topic_info=            Term         Freq        Total Category  logprob  loglift
698    emissions  5725.000000  5725.000000  Default  30.0000  30.0000
714       energy  5590.000000  5590.000000  Default  29.0000  29.0000
342         city  4898.000000  4898.000000  Default  28.0000  28.0000
346      climate  4302.000000  4302.000000  Default  27.0000  27.0000
1543        plan  3654.000000  3654.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
2371        year     8.608361  1423.469467  Topic10  -5.7371  -0.0733
1628    programs     8.905496  1551.013513  Topic10  -5.7032  -0.1252
489         cost     7.819086  1272.818996  Topic10  -5.8333  -0.0576
1724      reduce     8.017212  1665.642189  Topic10  -5.8083  -0.3016
380   commercial     7.529149  1278.353448  Topic10  -5.8711  -0.0997

[928 rows x 6 columns], token_table=       Topic      Freq             Term
term                                   
17064      1  0.443670  abidebytheretro
17064      2  0.443670  abidebytheretro
17         1  0.356388          achieve
17         2  0.229269          achieve
17         3  0.111229          achieve
...      ...       ...              ...
2371       8  0.011943             year
2371       9  0.008430             year
2371      10  0.006323             year
15980      1  0.411374            yorke
15980      2  0.411374            yorke

[3332 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 5, 10, 6, 9, 3, 7, 4, 1, 2])

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Experiment with the parameters (and perhaps the list of words that you exclude), and see if you can come up with sensible topics.
</div>

<div class="alert alert-block alert-info">
<h3>Key Takeaways</h3>
<ul>
  <li>Topic modeling is a form of dimensionality reduction. The aim is to make your data easier to interpret.</li>
  <li>Exploration and iteration are the keys. After your first set of results, maybe there are texts or words that you want to exclude. Then, experiment with adjusting the parameters.</li>
    <li>Success is defined based on whether you find the results useful!</li>
</ul>
</div>